# Example of Loading and pre-processing data for causality project

#### Using Iris for NetCDF data load and processing; Using xclim (and xarray) for calculating the maximum number of consecutive dry days (MCDD) using daily rainfall simulated by an ensemble of CMIP6 models (based on SSP245 and SSP585 scenarios) as input data

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.cm as mpl_cm

import xarray as xr

import iris
import iris.coord_categorisation as coord_cat
import iris.plot as iplt
import iris.quickplot as qplt
import cf_units

import xclim

C:\Users\user\Anaconda3\envs\xclim_env\lib\site-packages\pyproj\__init__.py:89: UserWarning: pyproj unable to set database path.
  _pyproj_global_context_initialize()


In [4]:
# Calculate maximum number of consecutive dry days for each model and save it
# CMIP6 daily rainfall input data retrieved from JASMIN HPC - https://help.ceda.ac.uk/article/4801-cmip6-data

DATA_PATH = '/CMIP6/pr/'

models = ['ACCESS-CM', 'ACCESS-ESM', 'CanESM',
          'CNRM-CM', 'CNRM-ESM', 'FGOALS-g3', 
          'GFDL', 'HadGEM3-GC31-LL', 'INM-CM4', 'INM-CM5', 
          'IPSL', 'MIROC', 'MPI-ESM1-2-LR', 'MRI', 'NESM', 'NorESM2-LM',
          'NorESM2-MM', 'TaiESM', 'UKESM', 'CMCC-CM', 'CMCC-ESM', 'KIOST']

scenarios = ['ssp245_nf', 'ssp245_ff', 'ssp585_nf', 'ssp585_ff']

for model in models[0:]:   # remove the '2' for all models
    for scenario in scenarios[0:]:
        print(model)
        print (scenario)
        path = DATA_PATH + model + '/'
        precip_path = xr.open_mfdataset(path + 'pr_day_' + model + '_' + scenario + '_daily.nc') # For SSP245 data from 2021-2060
        s_Borneo_jja = xclim.core.calendar.select_time (precip_path.pr, season = 'JJA')
        s_Borneo_jja = s_Borneo_jja.fillna(1)
        max_dry_days = xclim.indicators.atmos.maximum_consecutive_dry_days(pr = s_Borneo_jja, thresh = '1 mm/day') # Maximum consecutive dry days
        max_dry_days.to_netcdf (path + '' + model + '_consec_dry_days_' + scenario + '.nc')

ACCESS-CM
ACCESS-ESM


C:\Users\user\Anaconda3\envs\xclim_env\lib\site-packages\dask\array\reductions.py:608: RuntimeWarning: All-NaN slice encountered
  return np.nanmax(x_chunk, axis=axis, keepdims=keepdims)


CanESM
CNRM-CM
CNRM-ESM
FGOALS-g3


C:\Users\user\Anaconda3\envs\xclim_env\lib\site-packages\dask\array\reductions.py:608: RuntimeWarning: All-NaN slice encountered
  return np.nanmax(x_chunk, axis=axis, keepdims=keepdims)


GFDL
HadGEM3-GC31-LL
INM-CM4


...

C:\Users\user\Anaconda3\envs\xclim_env\lib\site-packages\dask\array\reductions.py:608: RuntimeWarning: All-NaN slice encountered
  return np.nanmax(x_chunk, axis=axis, keepdims=keepdims)


TaiESM


C:\Users\user\Anaconda3\envs\xclim_env\lib\site-packages\dask\array\reductions.py:608: RuntimeWarning: All-NaN slice encountered
  return np.nanmax(x_chunk, axis=axis, keepdims=keepdims)


UKESM


C:\Users\user\Anaconda3\envs\xclim_env\lib\site-packages\dask\array\reductions.py:608: RuntimeWarning: All-NaN slice encountered
  return np.nanmax(x_chunk, axis=axis, keepdims=keepdims)


In [10]:
# Calculate long-term mean and concatenate into time series

models = ['ACCESS-CM', 'ACCESS-ESM', 'CanESM',
          'CNRM-CM', 'CNRM-ESM', 'FGOALS-g3', 
          'GFDL', 'HadGEM3-GC31-LL', 'INM-CM4', 'INM-CM5', 
          'IPSL', 'MIROC', 'MPI-ESM1-2-LR', 'MRI', 'NESM', 'NorESM2-LM',
          'NorESM2-MM', 'TaiESM', 'UKESM', 'CMCC-CM', 'CMCC-ESM', 'KIOST']

scenarios = ['ssp245_nf', 'ssp245_ff', 'ssp585_nf', 'ssp585_ff']

for model in models[0:]:
    for scenario in scenarios[0:]:
        print(model)
        print (scenario)
        path = DATA_PATH + model + '/'
        max_dry_days_path = iris.load (path + model + '_consec_dry_days_' + scenario + '.nc')
        max_dry_days = max_dry_days_path[0]
        max_dry_days.coord('lat').guess_bounds()
        max_dry_days.coord('lon').guess_bounds()
        max_dry_days_ltm = max_dry_days.collapsed('time',
                                    iris.analysis.MEAN)
        # Perform the area-weighted mean using the computed grid-box areas.
        max_dry_days_mean = max_dry_days_ltm.collapsed(['lat', 'lon'],
                                        iris.analysis.MEAN)
        print (max_dry_days_mean.data)
        # Perform the area-weighted mean using the computed grid-box areas.
        max_dry_days_mean = max_dry_days.collapsed(['lat', 'lon'],
                                    iris.analysis.MEAN)
        iris.save (max_dry_days_mean, DATA_PATH + model + '/Borneo_consec_drydays_' + model + '_' + scenario + '.nc')

CanESM
ssp245_nf
12.0
CanESM
ssp245_ff
11.380555555555556


C:\Users\user\Anaconda3\envs\xclim_env\lib\site-packages\iris\coords.py:2220: UserWarning: Collapsing a non-contiguous coordinate. Metadata may not be fully descriptive for 'time'.
  warnings.warn(msg.format(self.name()))
C:\Users\user\Anaconda3\envs\xclim_env\lib\site-packages\iris\coords.py:2220: UserWarning: Collapsing a non-contiguous coordinate. Metadata may not be fully descriptive for 'time'.
  warnings.warn(msg.format(self.name()))
C:\Users\user\Anaconda3\envs\xclim_env\lib\site-packages\iris\coords.py:2220: UserWarning: Collapsing a non-contiguous coordinate. Metadata may not be fully descriptive for 'time'.
  warnings.warn(msg.format(self.name()))


CanESM
ssp585_ff
21.73888888888889
CNRM-CM
ssp245_nf
11.21214285714286


C:\Users\user\Anaconda3\envs\xclim_env\lib\site-packages\iris\coords.py:2220: UserWarning: Collapsing a non-contiguous coordinate. Metadata may not be fully descriptive for 'time'.
  warnings.warn(msg.format(self.name()))
C:\Users\user\Anaconda3\envs\xclim_env\lib\site-packages\iris\coords.py:2220: UserWarning: Collapsing a non-contiguous coordinate. Metadata may not be fully descriptive for 'time'.
  warnings.warn(msg.format(self.name()))


CNRM-CM
ssp245_ff
11.474285714285713
CNRM-CM
ssp585_ff
12.037857142857142


...

C:\Users\user\Anaconda3\envs\xclim_env\lib\site-packages\iris\coords.py:2220: UserWarning: Collapsing a non-contiguous coordinate. Metadata may not be fully descriptive for 'time'.
  warnings.warn(msg.format(self.name()))
C:\Users\user\Anaconda3\envs\xclim_env\lib\site-packages\iris\coords.py:2220: UserWarning: Collapsing a non-contiguous coordinate. Metadata may not be fully descriptive for 'time'.
  warnings.warn(msg.format(self.name()))


UKESM
ssp245_nf
13.272142857142857
UKESM
ssp245_ff


C:\Users\user\Anaconda3\envs\xclim_env\lib\site-packages\iris\coords.py:2220: UserWarning: Collapsing a non-contiguous coordinate. Metadata may not be fully descriptive for 'time'.
  warnings.warn(msg.format(self.name()))
C:\Users\user\Anaconda3\envs\xclim_env\lib\site-packages\iris\coords.py:2220: UserWarning: Collapsing a non-contiguous coordinate. Metadata may not be fully descriptive for 'time'.
  warnings.warn(msg.format(self.name()))


18.04357142857143
UKESM
ssp585_ff
21.72357142857143


C:\Users\user\Anaconda3\envs\xclim_env\lib\site-packages\iris\coords.py:2220: UserWarning: Collapsing a non-contiguous coordinate. Metadata may not be fully descriptive for 'time'.
  warnings.warn(msg.format(self.name()))
